In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

from tqdm import tqdm
from mobilenetv3 import *
from mobilenetv2 import MobileNetV2
from mobilenetv1 import MobileNet

In [2]:
# small
# net_small = mobilenetv3(mode='small')
# state_dict = torch.load('mobilenetv3_small_67.4.pth.tar')
# net_small.load_state_dict(state_dict)

# large
# net_large = mobilenetv3(mode='large')
#net_large = mobilenetv3(mode='large')

#x = torch.randn(2,3,224,224)
#y = net_large(x)
#print(y.size())
#print(y)
import os 
os.system("rm ./weights/*")

0

In [3]:
#model = mobilenetv3(mode='large', n_class=2)
model = MobileNet(n_class=2)

In [4]:
'''normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])'''
#normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                 std=[1, 1, 1])

input_size = 224
batch_size = 16
n_worker = 4

train_dir = './traindir'
train_datasets = datasets.ImageFolder(
        train_dir, transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #normalize,
    ]))
train_dataloader = torch.utils.data.DataLoader(
    train_datasets,
    batch_size=batch_size, shuffle=True,
    num_workers=n_worker, pin_memory=True)

val_dir = './valdir'
val_datasets = datasets.ImageFolder(
        val_dir, transforms.Compose([
        transforms.Resize(int(input_size/0.875)),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        #normalize,
    ]))

val_dataloader = torch.utils.data.DataLoader(
    val_datasets,
    batch_size=batch_size, shuffle=False,
    num_workers=n_worker, pin_memory=True)

In [5]:
total_epoch = 101
learning_rate = 0.001
save_every = 10
log_every = 5

# --------------------训练过程---------------------------------
if torch.cuda.is_available():
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, 50, 0.1)
loss_func = nn.CrossEntropyLoss()

Loss_list = []
Accuracy_list = []

for epoch in range(total_epoch):
    #print(' ====== ====== ====== epoch {}'.format(epoch))
    # training-----------------------------
    train_loss = 0.
    train_acc = 0.
    model.train()
    for batch_x, batch_y in (train_dataloader):  #tqdm
        batch_x, batch_y = Variable(batch_x).cuda(), Variable(batch_y).cuda()
        out = model(batch_x)
        loss = loss_func(out, batch_y)
        train_loss += loss.item()  #loss.data[0]
        pred = torch.max(out, 1)[1]
        train_correct = (pred == batch_y).sum()
        train_acc += train_correct.item()  #data[0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #print('Train Loss: {:.6f}, Acc: {:.6f}'.format(train_loss / (len(
        #train_datasets)), train_acc / (len(train_datasets))))

    # evaluation--------------------------------
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for batch_x, batch_y in (val_dataloader):  #tqdm
        with torch.no_grad():
            batch_x, batch_y = Variable(batch_x).cuda(), Variable(batch_y).cuda()
            out = model(batch_x)
            loss = loss_func(out, batch_y)
            eval_loss += loss.item()  #.data[0]
            pred = torch.max(out, 1)[1]
            num_correct = (pred == batch_y).sum()
            eval_acc += num_correct.item()  #.data[0]
    #print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        #val_datasets)), eval_acc / (len(val_datasets))))

    Loss_list.append(eval_loss / (len(val_datasets)))
    Accuracy_list.append(100 * eval_acc / (len(val_datasets)))

    if epoch % save_every == 0:
        print('Save model at epoch %d'%epoch)
        # save model
        torch.save(model.state_dict(), 'weights/mobilenet_%d.pkl'%epoch)
        #torch.save(model, 'weights/mobilenetv2_%d.pkl'%epoch)

    if epoch % log_every == 0:
        print('epoch {}'.format(epoch))
        #print('epoch {}, Train Loss: {:.6f}'.format((epoch), train_loss / (len(trainDataset))))
        print('Train Loss: {:.6f}, Acc: {:.6f}'.format(train_loss / (len(
                train_datasets)), train_acc / (len(train_datasets))))
        print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
                val_datasets)), eval_acc / (len(val_datasets))))

    scheduler.step()

Save model at epoch 0
epoch 0
Train Loss: 0.021677, Acc: 0.825000
Test Loss: 0.131395, Acc: 0.500000
epoch 5
Train Loss: 0.003994, Acc: 0.980000
Test Loss: 0.056103, Acc: 0.725000
Save model at epoch 10
epoch 10
Train Loss: 0.005426, Acc: 0.965000
Test Loss: 0.004147, Acc: 0.975000


KeyboardInterrupt: 

In [15]:
from PIL import Image
import numpy as np

transf = transforms.Compose([
         transforms.Resize(256),
         transforms.CenterCrop(224),
         transforms.ToTensor(),
         #transforms.Normalize(mean=[0.485, 0.456, 0.406],
         #                     std=[0.229, 0.224, 0.225]),
         ])

def image_loader(image_name):
    image = Image.open(image_name).convert('RGB')
    image = transf(image).unsqueeze(0)
    #print(np.array(image)[0][0][0][:10])
    return image.cuda()

def forward(image_name):
    inp = image_loader(image_name)
    model.eval()
    with torch.no_grad():
        out = model(inp).cpu()
    print(out)


In [16]:
forward("valdir/greenlight/000771_4.jpg")
forward("valdir/greenlight/000774_3.jpg")
forward("valdir/greenlight/000849_2.jpg")
forward("valdir/greenlight/000840_2.jpg")
forward("valdir/pointer/000650_1.jpg")
forward("valdir/pointer/000656_1.jpg")
forward("valdir/pointer/000692_2.jpg")
forward("valdir/pointer/000722_1.jpg")

tensor([[ 1.7085, -1.6978]])
tensor([[ 1.9104, -1.9086]])
tensor([[ 0.6705, -0.6266]])
tensor([[ 1.8742, -1.8742]])
tensor([[-3.5147,  3.2813]])
tensor([[-3.7945,  3.5505]])
tensor([[-4.2304,  3.9035]])
tensor([[-2.9786,  2.7847]])


In [73]:
ttt = Image.open("valdir/greenlight/000771_4.jpg").convert('RGB')

trans  = transforms.Compose([
         transforms.Resize(224),
         transforms.CenterCrop(224),
         transforms.ToTensor(),
         ])
ttt = trans(ttt)
print(np.array(ttt))

[[[0.45490196 0.43529412 0.43137255 ... 0.45490196 0.45490196 0.4627451 ]
  [0.46666667 0.44313726 0.43137255 ... 0.4745098  0.46666667 0.4745098 ]
  [0.45882353 0.4392157  0.42745098 ... 0.47843137 0.4745098  0.4862745 ]
  ...
  [0.1254902  0.09411765 0.0627451  ... 0.07058824 0.10196079 0.07450981]
  [0.1254902  0.09411765 0.05882353 ... 0.07058824 0.11372549 0.08627451]
  [0.11764706 0.08235294 0.05490196 ... 0.07450981 0.1254902  0.10196079]]

 [[0.49803922 0.47843137 0.4745098  ... 0.5137255  0.5137255  0.52156866]
  [0.50980395 0.4862745  0.4745098  ... 0.53333336 0.5254902  0.53333336]
  [0.5019608  0.48235294 0.47058824 ... 0.5372549  0.53333336 0.54509807]
  ...
  [0.1254902  0.10196079 0.07843138 ... 0.14509805 0.17254902 0.14117648]
  [0.1254902  0.09411765 0.07450981 ... 0.14117648 0.18431373 0.15686275]
  [0.11764706 0.08627451 0.07058824 ... 0.14117648 0.19607843 0.17254902]]

 [[0.4745098  0.45490196 0.44705883 ... 0.49411765 0.49411765 0.5019608 ]
  [0.4862745  0.462745